In [65]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, MSADataGrabberCSV, DataGrabberCSV
import pandas as pd
import numpy as np
import requests
from us import states

root = find_repo_root()

data = DataGrabberCSV()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

dataMA = MSADataGrabberCSV()
dataMA.get_features_wide(["gdp_ma"])
gdp_ma = dataMA.wide["gdp_ma"]


# note that there is also a library for accessing the Census data:
# https://github.com/datamade/census

In [66]:
# age variables, firstly let's focus just on the total population
# ,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E

variables =  "NAME,S0101_C01_001E"
county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
year = 2021

# interval = list(range(2010, 2023)) # it is possible to create this variable as a time series (only for ACS 5 year estimates)

dfs = []

for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
    fips = states.STATES[x].fips

    url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for=tract:{tract}&in=state:{fips}&in=county{county_fips}&key={api_key}'

    response = requests.get(url)

    assert response.status_code == 200, 'The data retrieval went wrong' # 200 means success

    print(f'{fips} fips done')

    data = response.json()

    df = pd.DataFrame(data[1:], columns=data[0])

    dfs.append(df)


combined_df = pd.concat(dfs, ignore_index=True)


01 fips done
02 fips done
04 fips done
05 fips done
06 fips done
08 fips done
09 fips done
10 fips done
12 fips done
13 fips done
15 fips done
16 fips done
17 fips done
18 fips done
19 fips done
20 fips done
21 fips done
22 fips done
23 fips done
24 fips done
25 fips done
26 fips done
27 fips done
28 fips done
29 fips done
30 fips done
31 fips done
32 fips done
33 fips done
34 fips done
35 fips done
36 fips done
37 fips done
38 fips done
39 fips done
40 fips done
41 fips done
42 fips done
44 fips done
45 fips done
46 fips done
47 fips done
48 fips done
49 fips done
50 fips done
51 fips done
53 fips done
54 fips done
55 fips done
56 fips done


In [99]:
combined_df.shape[0]

84208

In [87]:
pop = combined_df.copy()

In [88]:
column_mapping = {
    'S0101_C01_001E': 'total_pop'
}

pop.rename(columns=column_mapping, inplace=True)
pop.head()

,NAME,total_pop,state,county,tract
0,"Census Tract 201, Autauga County, Alabama",1791,01,001,020100
1,"Census Tract 202, Autauga County, Alabama",2010,01,001,020200
2,"Census Tract 203, Autauga County, Alabama",3577,01,001,020300
3,"Census Tract 204, Autauga County, Alabama",3802,01,001,020400
4,"Census Tract 205.01, Autauga County, Alabama",4381,01,001,020501


In [89]:
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

pop['GeoFIPS'] = pop['state'] + pop['county'] + pop['tract']
pop.drop(['state', 'county', 'tract'], axis=1, inplace=True)
pop['GeoFIPS'] = pop['GeoFIPS'].astype(np.int64)


pop['GeoName'] = pop['NAME'].apply(lambda x: f"{x.split(',')[1].strip().replace(' County', '')}, {state_abbreviations[x.split(',')[2].strip()]} (CT)")
pop = pop.drop(['NAME'], axis=1)


pop.head()

,total_pop,GeoFIPS,GeoName
0,1791,1001020100,"Autauga, AL (CT)"
1,2010,1001020200,"Autauga, AL (CT)"
2,3577,1001020300,"Autauga, AL (CT)"
3,3802,1001020400,"Autauga, AL (CT)"
4,4381,1001020501,"Autauga, AL (CT)"


In [94]:
len(pop['GeoName'].unique())

3142

In [97]:
pop.sort_values(by=['GeoFIPS', 'GeoName'], inplace=True)
pop = pop[['GeoFIPS', 'GeoName', 'total_pop']]

pop.head()

,GeoFIPS,GeoName,total_pop
0,1001020100,"Autauga, AL (CT)",1791
1,1001020200,"Autauga, AL (CT)",2010
2,1001020300,"Autauga, AL (CT)",3577
3,1001020400,"Autauga, AL (CT)",3802
4,1001020501,"Autauga, AL (CT)",4381


In [85]:
gdp

,GeoFIPS,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1001,"Autauga, AL",59.839,61.996,63.508,73.730,75.307,80.459,81.836,73.870,...,86.679,93.060,93.155,99.931,104.090,99.798,100.854,97.233,96.115,94.638
1,1003,"Baldwin, AL",73.853,77.273,81.570,90.523,101.402,104.553,107.840,102.635,...,99.000,104.651,106.431,110.434,115.476,118.498,125.068,131.431,131.614,144.294
2,1005,"Barbour, AL",113.864,111.853,114.628,124.473,125.004,122.611,118.397,110.695,...,103.918,113.335,106.760,103.702,101.969,100.509,101.804,102.053,98.044,99.393
3,1007,"Bibb, AL",80.443,81.527,85.124,89.317,88.782,89.597,95.308,94.745,...,102.559,99.537,97.933,94.594,95.812,96.878,96.988,104.620,109.487,107.878
4,1009,"Blount, AL",92.104,92.593,95.469,98.129,100.918,97.428,96.720,97.077,...,91.938,99.318,101.584,106.506,98.394,104.331,109.560,106.565,100.422,113.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,56037,"Sweetwater, WY",92.340,81.094,83.237,84.315,89.539,103.241,109.911,112.520,...,103.317,99.506,96.656,93.916,91.543,89.449,86.267,88.195,80.164,74.912
3067,56039,"Teton, WY",81.931,84.886,85.316,86.359,88.884,96.508,98.646,97.627,...,95.810,102.194,110.461,112.837,114.846,119.415,121.268,123.476,125.412,155.522
3068,56041,"Uinta, WY",99.906,110.550,107.053,108.929,112.808,123.552,126.916,127.784,...,106.181,96.674,94.344,95.302,94.681,93.024,89.431,86.759,77.512,76.147
3069,56043,"Washakie, WY",92.187,91.281,95.083,95.411,91.317,102.153,95.987,106.530,...,107.934,101.335,104.043,106.056,94.451,89.130,89.160,92.316,91.539,92.003


In [62]:
gdp_ma

,GeoFIPS,GeoName,2017,2018,2019,2020,2021
0,10180,"Abilene, TX (MA)",6827152.0,7141273.0,7426609.0,7354374.0,7573774.0
1,10420,"Akron, OH (MA)",36424857.0,36883382.0,37471099.0,36099097.0,37510049.0
2,10500,"Albany, GA (MA)",5693673.0,5702355.0,5839867.0,5797644.0,6068572.0
3,10540,"Albany-Lebanon, OR (MA)",4505965.0,4802294.0,4921026.0,4788775.0,4893295.0
4,10580,"Albany-Schenectady-Troy, NY (MA)",61024034.0,61119269.0,64250467.0,62574116.0,66759654.0
...,...,...,...,...,...,...,...
379,49420,"Yakima, WA (MA)",9986455.0,10545549.0,10696037.0,10523373.0,10676322.0
380,49620,"York-Hanover, PA (MA)",21304241.0,21335230.0,21554288.0,20545120.0,21095422.0
381,49660,"Youngstown-Warren-Boardman, OH-PA (MA)",20570339.0,20514155.0,20930410.0,19772228.0,20398971.0
382,49700,"Yuba City, CA (MA)",6342091.0,6679728.0,6781174.0,6740300.0,7014656.0


In [63]:
gdp

,GeoFIPS,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1001,"Autauga, AL",59.839,61.996,63.508,73.730,75.307,80.459,81.836,73.870,...,86.679,93.060,93.155,99.931,104.090,99.798,100.854,97.233,96.115,94.638
1,1003,"Baldwin, AL",73.853,77.273,81.570,90.523,101.402,104.553,107.840,102.635,...,99.000,104.651,106.431,110.434,115.476,118.498,125.068,131.431,131.614,144.294
2,1005,"Barbour, AL",113.864,111.853,114.628,124.473,125.004,122.611,118.397,110.695,...,103.918,113.335,106.760,103.702,101.969,100.509,101.804,102.053,98.044,99.393
3,1007,"Bibb, AL",80.443,81.527,85.124,89.317,88.782,89.597,95.308,94.745,...,102.559,99.537,97.933,94.594,95.812,96.878,96.988,104.620,109.487,107.878
4,1009,"Blount, AL",92.104,92.593,95.469,98.129,100.918,97.428,96.720,97.077,...,91.938,99.318,101.584,106.506,98.394,104.331,109.560,106.565,100.422,113.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,56037,"Sweetwater, WY",92.340,81.094,83.237,84.315,89.539,103.241,109.911,112.520,...,103.317,99.506,96.656,93.916,91.543,89.449,86.267,88.195,80.164,74.912
3067,56039,"Teton, WY",81.931,84.886,85.316,86.359,88.884,96.508,98.646,97.627,...,95.810,102.194,110.461,112.837,114.846,119.415,121.268,123.476,125.412,155.522
3068,56041,"Uinta, WY",99.906,110.550,107.053,108.929,112.808,123.552,126.916,127.784,...,106.181,96.674,94.344,95.302,94.681,93.024,89.431,86.759,77.512,76.147
3069,56043,"Washakie, WY",92.187,91.281,95.083,95.411,91.317,102.153,95.987,106.530,...,107.934,101.335,104.043,106.056,94.451,89.130,89.160,92.316,91.539,92.003


Different year

In [98]:
# age variables, firstly let's focus just on the total population
# ,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E

variables =  "NAME,S0101_C01_001E"
county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
year = 2015

# interval = list(range(2010, 2023)) # it is possible to create this variable as a time series (only for ACS 5 year estimates)

dfs = []

for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
    fips = states.STATES[x].fips

    url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for=tract:{tract}&in=state:{fips}&in=county{county_fips}&key={api_key}'

    response = requests.get(url)

    assert response.status_code == 200, 'The data retrieval went wrong' # 200 means success

    print(f'{fips} fips done')

    data = response.json()

    df = pd.DataFrame(data[1:], columns=data[0])

    dfs.append(df)


combined_df2 = pd.concat(dfs, ignore_index=True)


01 fips done
02 fips done
04 fips done
05 fips done
06 fips done
08 fips done
09 fips done
10 fips done
12 fips done
13 fips done
15 fips done
16 fips done
17 fips done
18 fips done
19 fips done
20 fips done
21 fips done
22 fips done
23 fips done
24 fips done
25 fips done
26 fips done
27 fips done
28 fips done
29 fips done
30 fips done
31 fips done
32 fips done
33 fips done
34 fips done
35 fips done
36 fips done
37 fips done
38 fips done
39 fips done
40 fips done
41 fips done
42 fips done
44 fips done
45 fips done
46 fips done
47 fips done
48 fips done
49 fips done
50 fips done
51 fips done
53 fips done
54 fips done
55 fips done
56 fips done


In [100]:
combined_df2.shape[0]

72877